In [ ]:
# Copyright 2026 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI Model Garden - PaliGemma (Finetuning)

<table><tbody><tr>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fcommunity%2Fmodel_garden%2Fmodel_garden_jax_paligemma_finetuning.ipynb">
      <img alt="Google Cloud Colab Enterprise logo" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" width="32px"><br> Run in Colab Enterprise
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_jax_paligemma_finetuning.ipynb">
      <img alt="GitHub logo" src="https://github.githubassets.com/assets/GitHub-Mark-ea2971cee799.png" width="32px"><br> View on GitHub
    </a>
  </td>
</tr></tbody></table>

## Overview

This notebook demonstrates how to do finetuning PaliGemma with a Vertex AI Custom Training Job, deploying the finetuned model to a Vertex AI Endpoint, and making online predictions.


### Objective
- Prepare data for finetuning.
- Launch a Vertex AI Custom Training Job to finetune PaliGemma, storing the resulting model to a GCS bucket.
- Deploy the finetuned PaliGemma model to a Vertex AI Endpoint.
- Make predictions to the endpoint.

### Costs

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage

Learn about [Vertex AI pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage pricing](https://cloud.google.com/storage/pricing), and use the [Pricing Calculator](https://cloud.google.com/products/calculator/) to generate a cost estimate based on your projected usage.

## Before you begin

In [ ]:
# @title Setup Google Cloud project
# @markdown 1. [Make sure that billing is enabled for your project](https://cloud.google.com/billing/docs/how-to/modify-project).

# @markdown 2. For finetuning, **[click here](https://console.cloud.google.com/iam-admin/quotas?location=us-central1&metric=aiplatform.googleapis.com%2Frestricted_image_training_nvidia_a100_80gb_gpus)** to check if your project already has the required 8 Nvidia A100 80 GB GPUs in the us-central1 region. If yes, then run this notebook in the us-central1 region. If you do not have 8 Nvidia A100 80 GPUs or have more GPU requirements than this, then schedule your job with Nvidia H100 GPUs via Dynamic Workload Scheduler using [these instructions](https://cloud.google.com/vertex-ai/docs/training/schedule-jobs-dws). For Dynamic Workload Scheduler, check the [us-central1](https://console.cloud.google.com/iam-admin/quotas?location=us-central1&metric=aiplatform.googleapis.com%2Fcustom_model_training_preemptible_nvidia_h100_gpus) or [europe-west4](https://console.cloud.google.com/iam-admin/quotas?location=europe-west4&metric=aiplatform.googleapis.com%2Fcustom_model_training_preemptible_nvidia_h100_gpus) quota for Nvidia H100 GPUs. If you do not have enough GPUs, then you can follow [these instructions](https://cloud.google.com/docs/quotas/view-manage#viewing_your_quota_console) to request quota.

# @markdown 3. For serving, **[click here](https://console.cloud.google.com/iam-admin/quotas?location=us-central1&metric=aiplatform.googleapis.com%2Fcustom_model_serving_nvidia_l4_gpus)** to check if your project already has the required 1 L4 GPU in the us-central1 region.  If yes, then run this notebook in the us-central1 region. If you need more L4 GPUs for your project, then you can follow [these instructions](https://cloud.google.com/docs/quotas/view-manage#viewing_your_quota_console) to request more. Alternatively, if you want to run predictions with A100 80GB or H100 GPUs, we recommend using the regions listed below. **NOTE:** Make sure you have associated quota in selected regions. Click the links to see your current quota for each GPU type: [Nvidia A100 80GB](https://console.cloud.google.com/iam-admin/quotas?metric=aiplatform.googleapis.com%2Fcustom_model_serving_nvidia_a100_80gb_gpus), [Nvidia H100 80GB](https://console.cloud.google.com/iam-admin/quotas?metric=aiplatform.googleapis.com%2Fcustom_model_serving_nvidia_h100_gpus).

# @markdown > | Machine Type | Accelerator Type | Recommended Regions |
# @markdown | ----------- | ----------- | ----------- |
# @markdown | a2-ultragpu-1g | 1 NVIDIA_A100_80GB | us-central1, us-east4, europe-west4, asia-southeast1, us-east4 |
# @markdown | a3-highgpu-2g | 2 NVIDIA_H100_80GB | us-west1, asia-southeast1, europe-west4 |
# @markdown | a3-highgpu-4g | 4 NVIDIA_H100_80GB | us-west1, asia-southeast1, europe-west4 |
# @markdown | a3-highgpu-8g | 8 NVIDIA_H100_80GB | us-central1, europe-west4, us-west1, asia-southeast1 |

# @markdown 4. **[Optional]** [Create a Cloud Storage bucket](https://cloud.google.com/storage/docs/creating-buckets) for storing experiment outputs. Set the BUCKET_URI for the experiment environment. The specified Cloud Storage bucket (`BUCKET_URI`) should be located in the same region as where the notebook was launched. Note that a multi-region bucket (eg. "us") is not considered a match for a single region covered by the multi-region range (eg. "us-central1"). If not set, a unique GCS bucket will be created instead.

BUCKET_URI = "gs://"  # @param {type:"string"}

# @markdown 5. **[Optional]** Set region. If not set, the region will be set automatically according to Colab Enterprise environment.

REGION = ""  # @param {type:"string"}

! git clone https://github.com/GoogleCloudPlatform/vertex-ai-samples.git

import datetime
import importlib
import json
import os
import tempfile
import uuid
from typing import Tuple

import matplotlib.pyplot as plt
from google.cloud import aiplatform

common_util = importlib.import_module(
    "vertex-ai-samples.notebooks.community.model_garden.docker_source_codes.notebook_util.common_util"
)

models, endpoints = {}, {}

# Get the default cloud project id.
PROJECT_ID = os.environ["GOOGLE_CLOUD_PROJECT"]

# Get the default region for launching jobs.
if not REGION:
    if not os.environ.get("GOOGLE_CLOUD_REGION"):
        raise ValueError(
            "REGION must be set. See"
            " https://cloud.google.com/vertex-ai/docs/general/locations for"
            " available cloud locations."
        )
    REGION = os.environ["GOOGLE_CLOUD_REGION"]

# Enable the Vertex AI API and Compute Engine API, if not already.
print("Enabling Vertex AI API and Compute Engine API.")
! gcloud services enable aiplatform.googleapis.com compute.googleapis.com

# Cloud Storage bucket for storing the experiment artifacts.
# A unique GCS bucket will be created for the purpose of this notebook. If you
# prefer using your own GCS bucket, change the value yourself below.
now = datetime.datetime.now().strftime("%Y%m%d%H%M%S")
BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])

if BUCKET_URI is None or BUCKET_URI.strip() == "" or BUCKET_URI == "gs://":
    BUCKET_URI = f"gs://{PROJECT_ID}-tmp-{now}-{str(uuid.uuid4())[:4]}"
    BUCKET_NAME = "/".join(BUCKET_URI.split("/")[:3])
    ! gsutil mb -l {REGION} {BUCKET_URI}
else:
    assert BUCKET_URI.startswith("gs://"), "BUCKET_URI must start with `gs://`."
    shell_output = ! gsutil ls -Lb {BUCKET_NAME} | grep "Location constraint:" | sed "s/Location constraint://"
    bucket_region = shell_output[0].strip().lower()
    if bucket_region != REGION:
        raise ValueError(
            "Bucket region %s is different from notebook region %s"
            % (bucket_region, REGION)
        )
print(f"Using this GCS Bucket: {BUCKET_URI}")

STAGING_BUCKET = os.path.join(BUCKET_URI, "temporal")
MODEL_BUCKET = os.path.join(BUCKET_URI, "paligemma")


# Initialize Vertex AI API.
print("Initializing Vertex AI API.")
aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=STAGING_BUCKET)

# Gets the default SERVICE_ACCOUNT.
shell_output = ! gcloud projects describe $PROJECT_ID
project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"
print("Using this default Service Account:", SERVICE_ACCOUNT)


# Provision permissions to the SERVICE_ACCOUNT with the GCS bucket
! gsutil iam ch serviceAccount:{SERVICE_ACCOUNT}:roles/storage.admin $BUCKET_NAME

! gcloud config set project $PROJECT_ID
! gcloud projects add-iam-policy-binding --no-user-output-enabled {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role="roles/storage.admin"
! gcloud projects add-iam-policy-binding --no-user-output-enabled {PROJECT_ID} --member=serviceAccount:{SERVICE_ACCOUNT} --role="roles/aiplatform.user"

pretrained_filename_lookup = {
    "paligemma-224-float32": "pt_224.npz",
    "paligemma-448-float32": "pt_448.npz",
    "paligemma-896-float32": "pt_896.npz",
    "paligemma-mix-224-float32": "mix_224.npz",
    "paligemma-mix-448-float32": "mix_448.npz",
}

In [ ]:
# @title Access PaliGemma models on Vertex AI for GPU based serving


# @markdown Accept the model agreement to access the models:
# @markdown 1. Open the [PaliGemma model card](https://console.cloud.google.com/vertex-ai/publishers/google/model-garden/363) from [Vertex AI Model Garden](https://cloud.google.com/model-garden).
# @markdown 1. Review and accept the agreement in the pop-up window on the model card page. If you have previously accepted the model agreement, there will not be a pop-up window on the model card page and this step is not needed.
# @markdown 1. After accepting the agreement of PaliGemma, a `gs://` URI containing PaliGemma pretrained models will be shared.
# @markdown 1. Paste the link in the `VERTEX_AI_MODEL_GARDEN_PALIGEMMA` field below.
# @markdown 1. The PaliGemma models will be copied into `BUCKET_URI`.
# @markdown The file transfer can take anywhere from 15 minutes to 30 minutes.
VERTEX_AI_MODEL_GARDEN_PALIGEMMA = "gs://"  # @param {type:"string", isTemplate:true}
assert (
    VERTEX_AI_MODEL_GARDEN_PALIGEMMA and VERTEX_AI_MODEL_GARDEN_PALIGEMMA != "gs://"
), "Click the agreement of PaliGemma in Vertex AI Model Garden, and get the GCS path of PaliGemma model artifacts."
print(
    "Copying PaliGemma model artifacts from",
    VERTEX_AI_MODEL_GARDEN_PALIGEMMA,
    "to ",
    MODEL_BUCKET,
)

! gsutil -m cp -R $VERTEX_AI_MODEL_GARDEN_PALIGEMMA/* $MODEL_BUCKET

assert (
    os.system(f"gsutil ls {MODEL_BUCKET}") == 0
), f"MODEL_BUCKET does not exist: {MODEL_BUCKET}."
model_path_prefix = MODEL_BUCKET

## Finetune with Vertex AI Custom Training Jobs

In [ ]:
# @title Data preparation

# @markdown The dataset file format is to be image-string pairs stored in a jsonl file.
# @markdown The value for `"image"` can be a GCS path or a URL.

# @markdown ```
# @markdown {"image": "gs://bucket-name/image.jpg", "prefix": "What animal is this?", "suffix": "cat"}
# @markdown {"image": "https://google.com/image.jpg", "prefix": "What drink is this?", "suffix": "soda"}
# @markdown ```

dataset_gcs_uri = "gs://longcap100/data_train90.jsonl"  # @param {type: "string"}

# @markdown [Optional] You can specify the `image` fields in the JSONL file to
# @markdown contain only filenames. In this case, you must also provide the
# @markdown image storage location in `dataset_image_dir`. If the JSONL file
# @markdown already contains full paths to the images, leave
# @markdown `dataset_image_dir` blank. Note that the `SERVICE_ACCOUNT` defined
# @markdown above must have read access to the images.
dataset_image_dir = ""  # @param {type:"string"}

# Set defaults for the example dataset.
if dataset_gcs_uri == "gs://longcap100/data_train90.jsonl" and not dataset_image_dir:
    dataset_image_dir = "gs://longcap100"

### Inference examples before finetuning

The images below are some of the examples of inference results of the pretrained
 `paligemma-224-float32` checkpoint.

| Image | URI | Caption |
|-----|-----|-----|
| <img src="https://storage.googleapis.com/longcap100/91.jpeg" width="200" >  | gs://longcap100/91.jpeg | the beauty of the sleeve |
| <img src="https://storage.googleapis.com/longcap100/92.jpeg" width="200" >   | gs://longcap100/92.jpeg | how to wear a maxi dress for summer |
| <img src="https://storage.googleapis.com/longcap100/93.jpeg" width="200" >  | gs://longcap100/93.jpeg | a red blazer and black bag , a key piece of the week 's fashion . |
| <img src="https://storage.googleapis.com/longcap100/94.jpeg" width="200" >   | gs://longcap100/94.jpeg | how to wear boyfriend jeans like a fashion blogger |
| <img src="https://storage.googleapis.com/longcap100/95.jpeg" width="200" >   | gs://longcap100/95.jpeg | this graphic sweatshirt is a must have for your wardrobe . |
| <img src="https://storage.googleapis.com/longcap100/96.jpeg" width="200" >   | gs://longcap100/96.jpeg | person in a long shot of our model |


### Finetune

In [ ]:
# @title Run
# @markdown Use the Vertex AI SDK to create and run the custom training jobs. With the default setting, you can train about 30 steps per minute with the batch size of 4.

model_variant = "pt"  # @param ["mix", "pt"]
model_resolution = 224  # @param [224, 448, 896]
model_precision_type = "float32"  # Only float32 is supported.

if model_variant == "mix":
    model_name_prefix = "paligemma-mix"
else:
    model_name_prefix = "paligemma"

base_model_name = f"{model_name_prefix}-{model_resolution}-{model_precision_type}"
base_model_filename = pretrained_filename_lookup[base_model_name]
base_model_uri = os.path.join(model_path_prefix, base_model_filename)

# The pre-built training docker image.
TRAIN_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/jax-paligemma-train-gpu:20240807_0916_RC00"

# The accelerator to use.
ACCELERATOR_TYPE = "NVIDIA_L4"  # @param ["NVIDIA_TESLA_V100", "NVIDIA_L4"]

# Batch size for finetuning.
batch_size = 4  # @param {type:"integer"}
# Number of epochs to train.
epochs = 3  # @param {type:"integer"}
# Learning rate.
learning_rate = 0.1  # @param{type:"number"}
# Text length.
text_length = 512  # @param{type:"integer"}

# Worker pool spec.

if ACCELERATOR_TYPE == "NVIDIA_TESLA_V100":
    machine_type = "n1-standard-8"
    accelerator_count = 2
elif ACCELERATOR_TYPE == "NVIDIA_L4":
    machine_type = "g2-standard-24"
    accelerator_count = 2
else:
    raise ValueError(
        f"Cannot automatically determine machine type from {ACCELERATOR_TYPE}."
    )

replica_count = 1

common_util.check_quota(
    project_id=PROJECT_ID,
    region=REGION,
    accelerator_type=ACCELERATOR_TYPE,
    accelerator_count=accelerator_count,
    is_for_training=True,
)

# Setup training job.
job_name = common_util.get_job_name_with_datetime("paligemma-finetune")

# Add labels for the finetuning job.
labels = {
    "mg-source": "notebook",
    "mg-notebook-name": "model_garden_jax_paligemma_finetuning.ipynb".split(".")[0],
}

labels["mg-tune"] = "publishers-google-models-paligemma"
versioned_model_id = base_model_name.lower().replace(".", "-")
labels["versioned-mg-tune"] = f"{labels['mg-tune']}-{versioned_model_id}"

# Pass training arguments and launch job.
train_job = aiplatform.CustomContainerTrainingJob(
    display_name=job_name,
    container_uri=TRAIN_DOCKER_URI,
    labels=labels,
)

# Designate a GCS folder to store the LORA adapter.
finetune_output_dir_name = common_util.get_job_name_with_datetime("paligemma-finetune")
finetune_output_dir = os.path.join(STAGING_BUCKET, finetune_output_dir_name)

train_args = [
    "--config=big_vision/configs/proj/paligemma/transfers/vertexai_l4.py",
    f"--workdir={finetune_output_dir}",
    f"--config.model_init={base_model_uri}",
    f"--config.input.data.fname={dataset_gcs_uri}",
    f"--config.text_len={text_length}",
]

if batch_size:
    train_args.append(f"--config.input.batch_size={batch_size}")
if epochs:
    train_args.append(f"--config.total_epochs={epochs}")
if learning_rate:
    train_args.append(f"--config.lr={learning_rate}")

train_args.append(f"--config.input.data.fopen_keys.image={dataset_image_dir}")
train_job.run(
    args=train_args,
    replica_count=replica_count,
    machine_type=machine_type,
    accelerator_type=ACCELERATOR_TYPE,
    accelerator_count=accelerator_count,
    boot_disk_size_gb=500,
    service_account=SERVICE_ACCOUNT,
)

print("Checkpoint and log files was saved in: ", finetune_output_dir)

# @markdown Click "Show Code" to see more details.

In [ ]:
# @title View training loss
# @markdown Metrics will be stored in a file named `big_vision_metrics.txt` in the GCS bucket, including training loss, examples seen, and core hours throughout training.

# @markdown Run this cell to get and plot the training loss.

# Get relevant metrics from metrics file.
metrics_file_name = "big_vision_metrics.txt"
metrics_path = os.path.join(finetune_output_dir, metrics_file_name)

temp_dir = tempfile.TemporaryDirectory()
local_metrics_path = os.path.join(temp_dir.name, metrics_file_name)

! gsutil cp $metrics_path $local_metrics_path

steps = []
training_losses = []
with open(local_metrics_path, "r") as f:
    for line in f:
        metric = json.loads(line)
        steps.append(metric["step"])
        training_losses.append(metric["training_loss"])

# Plot training plot
plt.plot(steps, training_losses)
plt.title("Steps vs. Training Loss")
plt.xlabel("Steps")
plt.ylabel("Training Loss")
plt.show()

## Deployment and prediction

In [ ]:
# @title Deploy
# @markdown This section uploads the finetuned PaliGemma model to Model Registry and deploys it to a Vertex AI Endpoint. It takes approximately 15 minutes to finish.

# @markdown Note: You cannot use accelerator type `NVIDIA_TESLA_V100` to serve prebuilt or finetuned PaliGemma models with resolution `896`.

last_checkpoint_file_name = "checkpoint.bv-LAST"
last_checkpoint_path = os.path.join(finetune_output_dir, last_checkpoint_file_name)

local_last_checkpoint_path = os.path.join(temp_dir.name, last_checkpoint_file_name)

! gsutil cp $last_checkpoint_path $local_last_checkpoint_path

with open(local_last_checkpoint_path, "r") as f:
    final_checkpoint_name = "checkpoint.bv-" + f.read()
    checkpoint_path = os.path.join(finetune_output_dir, final_checkpoint_name)

model_name = f"paligemma-{model_resolution}-{model_precision_type}-custom"
print(f"Deploying custom PaliGemma model: {model_name}")

# The pre-built serving docker images.
SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/jax-paligemma-serve-gpu:20240807_0916_RC00"

# @markdown Select the accelerator type to use to deploy the model:
accelerator_type = "NVIDIA_L4"  # @param ["NVIDIA_L4", "NVIDIA_TESLA_V100"]
if accelerator_type == "NVIDIA_L4":
    machine_type = "g2-standard-16"
    accelerator_count = 1
elif accelerator_type == "NVIDIA_TESLA_V100":
    if model_resolution == 896 and model_precision_type == "float32":
        raise ValueError(
            "NVIDIA_TESLA_V100 is not sufficient. Multi-gpu is not supported for PaLIGemma."
        )
    else:
        machine_type = "n1-highmem-8"
        accelerator_count = 1
else:
    raise ValueError(
        f"Recommended machine settings not found for: {accelerator_type}. To use another another accelerator, edit this code block to pass in an appropriate `machine_type`, `accelerator_type`, and `accelerator_count` to the deploy_model function by clicking `Show Code` and then modifying the code."
    )

# @markdown Set use_dedicated_endpoint to False if you don't want to use [dedicated endpoint](https://cloud.google.com/vertex-ai/docs/general/deployment#create-dedicated-endpoint). Note that [dedicated endpoint does not support VPC Service Controls](https://cloud.google.com/vertex-ai/docs/predictions/choose-endpoint-type), uncheck the box if you are using VPC-SC.
use_dedicated_endpoint = True  # @param {type:"boolean"}

common_util.check_quota(
    project_id=PROJECT_ID,
    region=REGION,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    is_for_training=False,
)


def deploy_model(
    model_name: str,
    checkpoint_path: str,
    machine_type: str = "g2-standard-32",
    accelerator_type: str = "NVIDIA_L4",
    accelerator_count: int = 1,
    resolution: int = 224,
    use_dedicated_endpoint: bool = False,
) -> Tuple[aiplatform.Model, aiplatform.Endpoint]:
    """Create a Vertex AI Endpoint and deploy the specified model to the endpoint."""
    model_name_with_time = common_util.get_job_name_with_datetime(model_name)
    endpoint = aiplatform.Endpoint.create(
        display_name=f"{model_name_with_time}-endpoint",
        dedicated_endpoint_enabled=use_dedicated_endpoint,
    )
    model = aiplatform.Model.upload(
        display_name=model_name_with_time,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[8080],
        serving_container_predict_route="/predict",
        serving_container_health_route="/health",
        serving_container_environment_variables={
            "CKPT_PATH": checkpoint_path,
            "RESOLUTION": resolution,
            "MODEL_ID": model_name,
            "DEPLOY_SOURCE": "notebook",
        },
        model_garden_source_model_name="publishers/google/models/paligemma",
    )
    print(
        f"Deploying {model_name_with_time} on {machine_type} with {accelerator_count} {accelerator_type} GPU(s)."
    )
    model.deploy(
        endpoint=endpoint,
        machine_type=machine_type,
        accelerator_type=accelerator_type,
        accelerator_count=accelerator_count,
        deploy_request_timeout=1800,
        service_account=SERVICE_ACCOUNT,
        enable_access_logging=True,
        min_replica_count=1,
        sync=True,
        system_labels={"NOTEBOOK_NAME": "model_garden_jax_paligemma_finetuning.ipynb"},
    )
    return model, endpoint


# @markdown If you want to use other accelerator types not listed above, then check other Vertex AI prediction supported accelerators and regions at https://cloud.google.com/vertex-ai/docs/predictions/configure-compute. You may need to manually set the `machine_type`, `accelerator_type`, and `accelerator_count` in the code by clicking `Show code` first.
models["model"], endpoints["endpoint"] = deploy_model(
    model_name=model_name,
    checkpoint_path=checkpoint_path,
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    resolution=model_resolution,
    use_dedicated_endpoint=use_dedicated_endpoint,
)

### Image captioning

In [ ]:
# @markdown This section uses the deployed PaliGemma model to caption and describe an image in a chosen language. Check how the caption has changed compared to the examples above.

caption_prompt = True

# @markdown <img src="https://storage.googleapis.com/longcap100/91.jpeg" width="400" >

image_url = "https://storage.googleapis.com/longcap100/91.jpeg"  # @param {type:"string"}

language_code = "en"  # @param {type: "string"}

image = common_util.download_image(image_url)
display(image)

# Make a prediction.
image_base64 = common_util.image_to_base64(image)

caption = common_util.caption_predict(
    endpoints["endpoint"],
    language_code,
    image,
    caption_prompt,
    use_dedicated_endpoint=use_dedicated_endpoint,
)

print("Caption: ", caption)
# @markdown Click "Show Code" to see more details.

## Clean up resources

In [ ]:
# Delete the train job.
train_job.delete()

# Clean up the temporary directory
temp_dir.cleanup()

# @markdown  Delete the experiment models and endpoints to recycle the resources
# @markdown  and avoid unnecessary continuous charges that may incur.

# Undeploy model and delete endpoint.
for endpoint in endpoints.values():
    endpoint.delete(force=True)

# Delete models.
for model in models.values():
    model.delete()

delete_bucket = False  # @param {type:"boolean"}
if delete_bucket:
    ! gsutil -m rm -r $BUCKET_NAME